### Import libraries

In [1]:
from consistent_plots import hist, hist2d
import matplotlib as mpl
mpl.rcParams['font.size'] = 14
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
from trsm import trsm_combos
import vector

In [5]:
# mpl.rcParams.keys()

In [22]:
def norm_hist(arr, bins=100):
    n, b = np.histogram(arr, bins=bins)
    x = (b[:-1] + b[1:]) / 2
    
    return n/n.max(), b, x

### Load model

In [13]:
from keras.models import model_from_json

In [15]:
from pickle import load

In [14]:
json_file = open('../../../models/classifier/reco/in_six-out_two/model/model_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('../../../models/classifier/reco/in_six-out_two/model/model_1.h5')

In [16]:
scaler = load(open('../../../models/classifier/reco/in_six-out_two/model/scaler_1.pkl', 'rb'))

### Open ROOT file

In [8]:
# filename = f'../../../signal/NanoAOD/NMSSM_XYH_YToHH_6b_MX_700_MY_400_reco_test_set.root'
filename = f'../../../signal/skimmed/...'
combos = trsm_combos(filename=filename)
combos.build_p4()
combos.nevents

-- [INFO] -- /eos/user/s/srosenzw/miniconda3/envs/work/lib/python3.8/runpy.py -- Opening ROOT file ../../../signal/NanoAOD/NMSSM_XYH_YToHH_6b_MX_700_MY_400_reco_test_set.root with columns
----------------------------------------------------------------------------------------------------
                                            TABLE COLUMNS                                           
----------------------------------------------------------------------------------------------------
jet_pt                            jet_eta                           jet_phi                           
jet_m                             jet_btag                          jet_idx                           
HX_b1_recojet_m                   HX_b1_recojet_pt                  HX_b1_recojet_eta                 
HX_b1_recojet_phi                 HX_b1_recojet_ptRegressed         HX_b2_recojet_m                   
HX_b2_recojet_pt                  HX_b2_recojet_eta                 HX_b2_recojet_phi            

100%|██████████| 52891/52891 [00:04<00:00, 12323.50it/s]


52891

### Load combinations of 7 total jets

In [10]:
# 7 choose 6 combinations
combos_7, evt_tag_7, combo_features = combos.nCk()

Input shape = (56525, 30)
Total events chosen: 8075


In [ ]:
test_features_7 = scaler.transform(combo_features)
scores_7 = loaded_model.predict(test_features_7)[:,0]
scores_7

In [23]:
fig, ax = plt.subplots()

n_7, b_7, x_7 = norm_hist(scores_7)
c_n_7, b_7, x_7 = norm_hist(scores_7[evt_tag_7])
w_n_7, b_7, x_7 = norm_hist(scores_7[~evt_tag_7])

print(len(scores_7))
print(len(scores_7[evt_tag_7]))
print(len(scores_7[~evt_tag_7]))

hist(ax, x_7, weights=n_7, bins=b_7, label='All combos')
hist(ax, x_7, weights=c_n_7, bins=b_7, label='Correct combos')
hist(ax, x_7, weights=w_n_7, bins=b_7, label='Incorrect combos')
ax.legend(fontsize='small', loc=9)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

56525
8075
48450


### Load combinations of 8 total jets

In [9]:
combos_8, evt_tag_8, combo_features_8 = combos.nCk(n=8)

Input shape = (105672, 30)
Total events chosen: 3774


In [17]:
test_features_8 = scaler.transform(combo_features_8)
scores_8 = loaded_model.predict(test_features_8)[:,0]
scores_8

In [25]:
fig, ax = plt.subplots()

n_8, b_8, x_8 = norm_hist(scores_8)
c_n_8, b_8, x_8 = norm_hist(scores_8[evt_tag_8])
w_n_8, b_8, x_8 = norm_hist(scores_8[~evt_tag_8])

print(len(scores_8))
print(len(scores_8[evt_tag_8]))
print(len(scores_8[~evt_tag_8]))

hist(ax, x_8, weights=n_8, bins=b_8, label='All combos')
hist(ax, x_8, weights=c_n_8, bins=b_8, label='Correct combos')
hist(ax, x_8, weights=w_n_8, bins=b_8, label='Incorrect combos')
ax.legend(fontsize='small', loc=9)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

105672
3774
101898


### Limited background

In [10]:
signal_p4 = combos.sgnl_p4
background_p4 = combos.bkgd_p4

sgnl_evt_p4 = combos.sgnl_evt_p4
bkgd_evt_p4 = combos.bkgd_evt_p4

sgnl_node_targets = np.concatenate((np.repeat(1, len(signal_p4)), np.repeat(0, len(background_p4))))
bkgd_node_targets = np.where(sgnl_node_targets == 1, 0, 1)
targets = np.column_stack((sgnl_node_targets, bkgd_node_targets))
print(targets.shape)

(16150, 2)


In [11]:
features = combos.construct_features()

-- [INFO] -- /eos/user/s/srosenzw/miniconda3/envs/work/lib/python3.8/runpy.py -- Preparing signal training example inputs.


KeyboardInterrupt: 

In [ ]:
combos.n_bkgd

In [12]:
fig, ax = plt.subplots()
ax.set_title('Swapped Jet Distribution')

n, b = np.histogram(combos.n_bkgd, bins=np.arange(10))
x = (b[:-1] + b[1:])/2

hist(ax, x, weights=n/n.max(), bins=np.arange(10))
ax.set_xlabel('Number of Swapped Jets')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
combos.n_bkgd == 1

array([ True,  True, False, ...,  True,  True,  True])

In [14]:
fig, ax = plt.subplots()
ax.set_title('Swapped Jet Distribution')

hist(ax, x, weights=n/n.max(), bins=np.arange(10))
ax.set_xlabel('Number of Swapped Jets')
ax.set_yscale('log')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
norm_features = scaler.transform(features)

In [ ]:
pt  = features[:,0:6]
eta = features[:,6:12]
phi = features[:,12:18]
m   = np.ones(pt.shape)*4

In [ ]:
p4_0 = vector.obj(pt=pt[:,0], eta=eta[:,0], phi=phi[:,0], mass=m[:,0])
p4_1 = vector.obj(pt=pt[:,1], eta=eta[:,1], phi=phi[:,1], mass=m[:,1])
p4_2 = vector.obj(pt=pt[:,2], eta=eta[:,2], phi=phi[:,2], mass=m[:,2])
p4_3 = vector.obj(pt=pt[:,3], eta=eta[:,3], phi=phi[:,3], mass=m[:,3])
p4_4 = vector.obj(pt=pt[:,4], eta=eta[:,4], phi=phi[:,4], mass=m[:,4])
p4_5 = vector.obj(pt=pt[:,5], eta=eta[:,5], phi=phi[:,5], mass=m[:,5])

In [ ]:
p4 = p4_0 + p4_1 + p4_2 + p4_3 + p4_4 + p4_5

In [ ]:
p4.mass

In [22]:
plt.close('all')

In [23]:
fig, ax = plt.subplots()
ax.set_title('Test Examples')

sgnl_mass = p4.mass[targets[:,0]==1]
bkgd_mass = p4.mass[targets[:,0]==0]

bins = np.linspace(300,900,60)

n_s, b_s = np.histogram(sgnl_mass, bins=bins)
n_b, b_b = np.histogram(bkgd_mass, bins=bins)
n, b = np.histogram(np.concatenate((sgnl_mass,bkgd_mass)), bins=bins)

x = (b_s[1:] + b_s[:-1])/2

_ = hist(ax, x, weights=n_s/n.max(), bins=bins, label='Signal')
_ = hist(ax, x, weights=n_b/n.max(), bins=bins, label='Background')
# _ = hist(ax, x, weights=n/n.max(), label='All')
ax.legend(loc=1, fontsize='small')
ax.set_xlabel(r'Invariant Mass [GeV]')
ax.set_ylabel('AU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'AU')

In [48]:
np.sum(combos.n_bkgd == 7)

3

In [50]:
len(combos.n_bkgd)

8075

In [53]:
fig, ax = plt.subplots()
ax.set_title('Test Examples')

sgnl_mass = p4.mass[targets[:,0]==1][combos.n_bkgd == 1]
bkgd_mass = p4.mass[targets[:,0]==0][combos.n_bkgd == 1]

bins = np.linspace(300,900,60)

n_s, b_s = np.histogram(sgnl_mass, bins=bins)
n_b, b_b = np.histogram(bkgd_mass, bins=bins)
n, b = np.histogram(np.concatenate((sgnl_mass,bkgd_mass)), bins=bins)

x = (b_s[1:] + b_s[:-1])/2

_ = hist(ax, x, weights=n_s/n.max(), bins=bins, label='Signal')
_ = hist(ax, x, weights=n_b/n.max(), bins=bins, label='Background')
# _ = hist(ax, x, weights=n/n.max(), label='All')
ax.legend(loc=1, fontsize='small')
ax.set_xlabel(r'Invariant Mass [GeV]')
ax.set_ylabel('AU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'AU')

In [24]:
b = np.linspace(300,900,60)
xdata = (b[1:] + b[:-1]) / 2
ydata = n_s / n_s.max()

In [27]:
# Fit the masses with a Gaussian
from lmfit.models import GaussianModel, SkewedGaussianModel

model = GaussianModel()

# create parameters with initial guesses:
params = model.make_params(center=700, amplitude=1, sigma=50)

result = model.fit(ydata, params, x=xdata)
print(result.fit_report())

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 46
    # data points      = 59
    # variables        = 3
    chi-square         = 0.04025920
    reduced chi-square = 7.1891e-04
    Akaike info crit   = -424.107292
    Bayesian info crit = -417.874680
[[Variables]]
    amplitude:  149.456340 +/- 1.52973291 (1.02%) (init = 1)
    center:     626.390487 +/- 0.71142335 (0.11%) (init = 700)
    sigma:      60.1944372 +/- 0.71142338 (1.18%) (init = 50)
    fwhm:       141.747065 +/- 1.67527399 (1.18%) == '2.3548200*sigma'
    height:     0.99053100 +/- 0.01013842 (1.02%) == '0.3989423*amplitude/max(1e-15, sigma)'
[[Correlations]] (unreported correlations are < 0.100)
    C(amplitude, sigma) =  0.577


In [28]:
result.params

name,value,standard error,relative error,initial value,min,max,vary,expression
amplitude,149.456340,1.52973291,(1.02%),1,-inf,inf,True,
center,626.390487,0.71142335,(0.11%),700,-inf,inf,True,
sigma,60.1944372,0.71142338,(1.18%),50,0.00000000,inf,True,
fwhm,141.747065,1.67527399,(1.18%),117.74100000000001,-inf,inf,False,2.3548200*sigma
height,0.99053100,0.01013842,(1.02%),0.007978846000000001,-inf,inf,False,"0.3989423*amplitude/max(1e-15, sigma)"


In [29]:
params = {}
for name,param in result.params.items():
    print(name,param.value, param.stderr)
    params[name] = {'value':param.value, 'error':param.stderr}

amplitude 149.45634014691495 1.5297329058158344
center 626.3904865142864 0.7114233534251908
sigma 60.19443717449696 0.7114233760858145
fwhm 141.74706454724895 1.6752739949850302
height 0.9905310006462548 0.010138415564615767


In [30]:
scores = loaded_model.predict(norm_features)

In [31]:
scores

array([[0.88108224, 0.11891768],
       [0.90760785, 0.09239213],
       [0.8779286 , 0.12207133],
       ...,
       [0.28498268, 0.7150174 ],
       [0.9543243 , 0.04567567],
       [0.9742431 , 0.02575688]], dtype=float32)

In [32]:
fig, ax = plt.subplots()
ax.set_title('Test Examples')

sgnl_scores = scores[:,0][targets[:,0]==1]
bkgd_scores = scores[:,0][targets[:,0]==0]

n_s, b_s = np.histogram(sgnl_scores, bins=np.linspace(0,1,101))
n_b, b_b = np.histogram(bkgd_scores, bins=np.linspace(0,1,101))

x = (b_s[1:] + b_s[:-1])/2

_ = hist(ax, x, weights=n_s/n_s.max(), label='Signal')
_ = hist(ax, x, weights=n_b/n_b.max(), label='Background')
ax.legend(loc=9)
ax.set_xlabel('Discriminator Score')
ax.set_ylabel('AU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'AU')

In [30]:
fig, ax = plt.subplots()

_ = hist2d(ax, sgnl_mass, sgnl_scores, xbins=np.linspace(500,800,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
fig, ax = plt.subplots()

_ = hist2d(ax, bkgd_mass, bkgd_scores, xbins=np.linspace(0,1000,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
score_bins = np.arange(0,1.01,0.01)

In [33]:
eff = np.array(())
for i in score_bins:
    eff = np.append(eff, np.sum(sgnl_scores > i)/len(sgnl_scores))

In [34]:
fig, ax = plt.subplots()
ax.set_title('Efficiency Curve')

ax.plot(score_bins, eff)
ax.set_ylabel('% Passing Signal')
ax.set_xlabel('Discriminator Cut')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Discriminator Cut')

In [31]:
# fig, ax = plt.subplots()

# sgnl_mass = p4.mass[targets[:,0]==1]
# pass_mass = p4.mass[scores[:,0] > 0.5]

# n_s, b_s = np.histogram(sgnl_mass, bins=np.linspace(200,1200,101))
# n_b, b_b = np.histogram(pass_mass, bins=np.linspace(200,1200,101))
# n, b = np.histogram(p4.mass, bins=np.linspace(200,1200,101))

# x = (b_s[1:] + b_s[:-1])/2

# _ = hist(ax, x, weights=n_s/n.max(), label='Signal')
# _ = hist(ax, x, weights=n_b/n.max(), label='Passing Score')
# _ = hist(ax, x, weights=n/n.max(), label='All')
# ax.legend(loc=1)
# ax.set_xlabel('Invariant Mass [GeV]')
# ax.set_ylabel('AU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'AU')

In [32]:
# fig, ax = plt.subplots()

# sgnl_mass = p4.mass[targets[:,0]==1]
# sgnl_pass_mask = (targets[:,0]==1) & (scores[:,0] > 0.5)
# sgnl_fail_mask = (targets[:,0]==1) & (scores[:,0] <= 0.5)
# fail_mass = p4.mass[sgnl_fail_mask]

# n_s, b_s = np.histogram(sgnl_mass, bins=np.linspace(200,1200,101))
# n_b, b_b = np.histogram(pass_mass, bins=np.linspace(200,1200,101))
# n_f, b_f = np.histogram(fail_mass, bins=np.linspace(200,1200,101))
# n, b = np.histogram(p4.mass, bins=np.linspace(200,1200,101))

# x = (b_s[1:] + b_s[:-1])/2

# _ = hist(ax, x, weights=n_s/n.max(), label='Signal')
# _ = hist(ax, x, weights=n_b/n.max(), label='Passing Score')
# _ = hist(ax, x, weights=n_f/n.max(), label='Failing Score')
# _ = hist(ax, x, weights=n/n.max(), label='All')
# ax.legend(loc=1)
# ax.set_xlabel('Invariant Mass [GeV]')
# ax.set_ylabel('AU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'AU')

In [42]:
# fig, ax = plt.subplots()

# sgnl_mass = p4.mass[targets[:,0]==1]
# bkgd_pass_mask = (targets[:,0]==0) & (scores[:,0] > 0.5)
# bkgd_fail_mask = (targets[:,0]==0) & (scores[:,0] <= 0.5)
# fail_bkgd = p4.mass[bkgd_fail_mask]
# pass_bkgd = p4.mass[bkgd_pass_mask]

# n_s, b_s = np.histogram(bkgd_mass, bins=np.linspace(200,1200,101))
# n_b, b_b = np.histogram(pass_bkgd, bins=np.linspace(200,1200,101))
# n_f, b_f = np.histogram(fail_bkgd, bins=np.linspace(200,1200,101))

# x = (b_s[1:] + b_s[:-1])/2

# _ = hist(ax, x, weights=n_s/n.max(), label='Signal')
# _ = hist(ax, x, weights=n_b/n.max(), label='Passing Score')
# _ = hist(ax, x, weights=n_f/n.max(), label='Failing Score')
# ax.legend(loc=1)
# ax.set_xlabel('Invariant Mass [GeV]')
# ax.set_ylabel('AU')

In [43]:
# fig, ax = plt.subplots()

# sgnl_mass = p4.mass[targets[:,0]==1]
# sgnl_pass_mask = (targets[:,0]==1) & (scores[:,0] > 0.5)
# sgnl_fail_mask = (targets[:,0]==1) & (scores[:,0] <= 0.5)
# bkgd_pass_mask = (targets[:,0]==0) & (scores[:,0] > 0.5)
# bkgd_fail_mask = (targets[:,0]==0) & (scores[:,0] <= 0.5)
# fail_sgnl = p4.mass[sgnl_fail_mask]
# pass_sgnl = p4.mass[sgnl_pass_mask]
# fail_bkgd = p4.mass[bkgd_fail_mask]
# pass_bkgd = p4.mass[bkgd_pass_mask]

# n_s, b_s = np.histogram(sgnl_mass, bins=np.linspace(200,1200,101))
# n_b, b_b = np.histogram(pass_sgnl, bins=np.linspace(200,1200,101))
# n_f, b_f = np.histogram(fail_sgnl, bins=np.linspace(200,1200,101))

# x = (b_s[1:] + b_s[:-1])/2

# _ = hist(ax, x, weights=n_s/n.max(), label='Signal')
# _ = hist(ax, x, weights=n_b/n.max(), label='Passing Score')
# _ = hist(ax, x, weights=n_f/n.max(), label='Failing Score')
# ax.legend(loc=1)
# ax.set_xlabel('Invariant Mass [GeV]')
# ax.set_ylabel('AU')